This notebook is referenced from https://www.kaggle.com/code/debarshichanda/understanding-mean-average-precision 

Edits are made to understand the concept. 

In [9]:
import numpy as np 

In [10]:
gt = np.array(['a', 'b', 'c', 'd', 'e'])

preds1 = np.array(['b', 'c', 'a', 'd', 'e'])
preds2 = np.array(['a', 'b', 'c', 'd', 'e'])
preds3 = np.array(['f', 'b', 'c', 'd', 'e'])
preds4 = np.array(['a', 'f', 'e', 'g', 'b'])
preds5 = np.array(['a', 'f', 'c', 'g', 'b'])
preds6 = np.array(['d', 'c', 'b', 'a', 'e'])

### Precision: 
- positively predicted positive elements / all positive predictions by model 
$$ P = { \#\ of\ correct\ predictions\over \#\ of\ all\ predictions  } = {TP \over (TP + FP)} $$

### Information Retrieval
$${\displaystyle {\text{P}}={\frac {|\{{\text{relevant documents}}\}\cap \{{\text{retrieved documents}}\}|}{|\{{\text{retrieved documents}}\}|}}}$$

Precision at cutoff `k`, `P(k)`, is simply the precision calculated by considering only the subset of your predictions from rank 1 through `k`. <br>
Take the top k recommendations and find it's precision with the ground truth.<br>

Example 1:
If `gt=[a,b,c,d,e]` and `pred=[b,c,a,d,e]` then for `P@1` we only take the first recommendation from `pred` i.e.`b` and find it's `precision` with the `gt`. 
$${\displaystyle {\text{P}}={\frac {|\{{\text{gt}}\}\cap \{{\text{pred[:1]}}\}|}{|\{{\text{pred[:1]}}\}|}}={\frac {\text{1}}{\text{1}}}}$$ 
<br>
Example 2:
If `gt=[a,b,c,d,e]` and `pred=[f,b,c,d,e]` then for `P@1` we only take the first recommendation from `pred` i.e.`f` and find it's `precision` with the `gt`. 

$${\displaystyle {\text{P}}={\frac {|\{{\text{gt}}\}\cap \{{\text{pred[:1]}}\}|}{|\{{\text{pred[:1]}}\}|}}={\frac {\text{0}}{\text{1}}}}$$
<br>
Example 3:
If `gt=[a,b,c,d,e]` and `pred=[a,f,e,g,b]` then for `P@2` we only take the top 2 recommendations from `pred` i.e.`[a,f]` and find it's `precision` with the `gt`. Intersection between the two sets is `1` since only `a` is present in the `gt`.

$${\displaystyle {\text{P}}={\frac {|\{{\text{gt}}\}\cap \{{\text{pred[:2]}}\}|}{|\{{\text{pred[:2]}}\}|}}={\frac {\text{1}}{\text{2}}}}$$
<br>

Some more Examples:

| true  | predicted   | k  | P(k) |
|:-:|:-:|:-:|:-:|
| [a, b, c, d, e]  | [b, c, a, d, e]   | 1  | 1.0  |
| [a, b, c, d, e]  | [a, b, c, d, e]   | 1  | 1.0  |
| [a, b, c, d, e]  | [f, b, c, d, e]   | 1  | 0.0  |
| [a, b, c, d, e]  | [a, f, e, g, b]   | 2  | $$1\over2$$  |
| [a, b, c, d, e]  | [a, f, c, g, b]   | 3  | $$2\over3$$  |
| [a, b, c, d, e]  | [d, c, b, a, e]   | 3  | $$3\over3$$  ||

In [11]:
np.intersect1d([1,2,3], [4,4,5])

array([], dtype=int32)

In [12]:
def precision_at_k(y_true, y_pred, k):
    intersected = np.intersect1d(y_true, y_pred[:k])
    return len(intersected)/k 

In [13]:
precision_at_k(gt, preds1, 1), precision_at_k(gt, preds1, 2) 

(1.0, 1.0)

In [16]:
scores = []

scores.append(precision_at_k(gt, preds1, k=1))
scores.append(precision_at_k(gt, preds2, k=1))
scores.append(precision_at_k(gt, preds3, k=1))
scores.append(precision_at_k(gt, preds4, k=2))
scores.append(precision_at_k(gt, preds5, k=3))
scores.append(precision_at_k(gt, preds6, k=3))

scores 

[1.0, 1.0, 0.0, 0.5, 0.6666666666666666, 1.0]

### Rel@K

- indicator function
- 1 if item at rank 'k' is relevant(correct) label

In [19]:
def rel_at_k(y_true, y_pred, k):
    if y_pred[k-1] in y_true:
        return 1
    else:
        return 0 

In [23]:
preds1[5-1]

'e'

In [20]:
scores = []

scores.append(rel_at_k(gt, preds1, k=1))
scores.append(rel_at_k(gt, preds2, k=1))
scores.append(rel_at_k(gt, preds3, k=1))
scores.append(rel_at_k(gt, preds4, k=2))
scores.append(rel_at_k(gt, preds5, k=3))
scores.append(rel_at_k(gt, preds6, k=3))

scores 

[1, 1, 0, 0, 1, 1]

## Average Precision at K

* Product of 'P@K' and 'rel(k)' 

$${1\over{{min(n,12)}}} {\sum_{k=1}^{min(n,12)}P(k) \times rel(k)}$$

In [25]:
def avg_precision_at_k(y_true, y_pred, k):
    ap = 0.0 
    for i in range(1, k+1):
        ap+= precision_at_k(y_true=y_true, y_pred=y_pred, k=k) * rel_at_k(y_true, y_pred, k)

    return ap/min(k, len(y_true))

In [32]:
scores = []

scores.append(avg_precision_at_k(gt, preds1, k=4))
scores.append(avg_precision_at_k(gt, preds2, k=4))
scores.append(avg_precision_at_k(gt, preds3, k=4))
scores.append(avg_precision_at_k(gt, preds4, k=4))
scores.append(avg_precision_at_k(gt, preds5, k=4))
scores.append(avg_precision_at_k(gt, preds6, k=4))

scores 

[1.0, 1.0, 0.75, 0.0, 0.0, 1.0]

## Mean Average Precision at K
Take mean of Average Precision for all the users

In [35]:
def mean_avg_precision_at_k(y_true, y_pred, k=12):
    print([avg_precision_at_k(gt, pred, k) for gt, pred in zip(y_true, y_pred)])
    return np.mean([avg_precision_at_k(gt, pred, k) for gt, pred in zip(y_true, y_pred)])

In [36]:
y_pred

array([['b', 'c', 'a', 'd', 'e'],
       ['a', 'b', 'c', 'd', 'e'],
       ['f', 'b', 'c', 'd', 'e'],
       ['a', 'f', 'e', 'g', 'b'],
       ['a', 'f', 'c', 'g', 'b'],
       ['d', 'c', 'b', 'a', 'e']], dtype='<U1')

In [39]:
y_true = np.array([gt, gt, gt, gt, gt, gt])
y_pred = np.array([preds1, preds2, preds3, preds4, preds5, preds6])

print(avg_precision_at_k(gt, preds1, k=4))
print(avg_precision_at_k(gt, preds2, k=4))
print(avg_precision_at_k(gt, preds3, k=4))
print(avg_precision_at_k(gt, preds4, k=4))
print(avg_precision_at_k(gt, preds5, k=4))
print(avg_precision_at_k(gt, preds6, k=4))

1.0
1.0
0.75
0.0
0.0
1.0


In [41]:
mean_avg_precision_at_k(y_true, y_pred, k=4)

[1.0, 1.0, 0.75, 0.0, 0.0, 1.0]


0.625